In [30]:
import os
import time

from FeatureEngineering import FeatureEngineering as fe
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [13]:
# in this code, we aren't splitting by recording frames per gesture, but rather by the gestures as a whole so if WAVE has 10 gesture recordings, 8 go to train and 2 to test



def create_dataframe_from_data(input_path: str):
    dataframes = []
    for file_name in os.listdir(input_path):
        if file_name.endswith(".npy"):
            file_path = os.path.join(input_path, file_name)

            data = np.load(file_path, allow_pickle=True)
            df = pd.DataFrame(data)

            gesture = file_name.split("_")[0]
            gesture_index = int(file_name.split("_")[1].split(".")[0])

            # add two new columns to df 
            df["gesture"] = gesture
            df["gesture_index"] = gesture_index

            dataframes.append(df)
            df.sort_values(by="frame", inplace=True)
    
    return pd.concat(dataframes, ignore_index=True) if len(dataframes) > 0 else ValueError("Dataframe is empty")





input_path = "C:\\Users\\Gen3r\\Documents\\capstone\\ml_model\\data\\data_3"

dataframe = create_dataframe_from_data(input_path)


In [14]:
def create_index_dict(df: pd.DataFrame):

    diction = {x: [] for x in np.unique(df["gesture"].values.tolist())}
    grouped = df.groupby("gesture_index")
    for gesture_index, gesture_data in grouped:
        gesture = np.unique(gesture_data["gesture"].values.tolist())[0]
        temp = diction[gesture]  + [gesture_index]
        diction.update({gesture:temp})

    return diction

def split_dataset(df: pd.DataFrame, target_label: str, additional_targets: list=None, train_ratio=0.8, test_ratio=0.2):
    
    assert train_ratio + test_ratio == 1.0, "ratios must sum to 1."

    gesture_index_dict = create_index_dict(dataframe)

    train_indices = []
    test_indices = []

    # Calculate number of indices for training set and split
    for gesture, indices in gesture_index_dict.items():
        n_train = math.ceil(len(indices) * train_ratio)
        train_indices.extend(indices[:n_train])
        test_indices.extend(indices[n_train:])

    grouped = dataframe.groupby("gesture_index")

    train_frames = []
    test_frames = []

    for idx in train_indices:
        train_frames.append(grouped.get_group(idx))
        
    for idx in test_indices:
        test_frames.append(grouped.get_group(idx))
    
    # Concatenate the dataframes to create the final train and test sets
    train_set = pd.concat(train_frames).reset_index(drop=True)
    test_set = pd.concat(test_frames).reset_index(drop=True)

    # Separate X and y
    X_train = train_set.drop(columns=[target_label])
    y_train = train_set[[target_label] + additional_targets] if additional_targets else train_set[[target_label]]
    X_test = test_set.drop(columns=[target_label])
    y_test = test_set[[target_label] + additional_targets] if additional_targets else test_set[[target_label]]

    return X_train, y_train, X_test, y_test



X_train, y_train, X_test, y_test = split_dataset(dataframe, "gesture", ["frame", "gesture_index"])



In [43]:
def augment_model(df: pd.DataFrame, noise_level=0.0, translation_vector=None, rotation_angle=0.0):
    df_augmented = df.copy()


    landmark_columns = [f"{col}" for col in df_augmented.columns if col.startswith(("hx", "hy", "hz", "px", "py", "pz", "lx", "ly", "lz", "rx", "ry", "rz"))]
    
    num_body_parts = ("h", "p", "l", "r")

    x_columns = [col for col in landmark_columns if any(col.startswith(f'{i}x') for i in num_body_parts)] # this way works because of how i is defined before hand... don't really know
    y_columns = [col for col in landmark_columns if any(col.startswith(f'{i}y') for i in num_body_parts)]
    z_columns = [col for col in landmark_columns if any(col.startswith(f'{i}z') for i in num_body_parts)]


    # Add noise
    if noise_level > 0:
        noise = np.random.normal(0, noise_level, df[x_columns + y_columns + z_columns].shape)
        df_augmented[x_columns + y_columns + z_columns] += noise

    # Apply translation
    if translation_vector is not None:
        for i, col in enumerate(x_columns):
            df_augmented[col] += translation_vector[i % 3]
        for i, col in enumerate(y_columns):
            df_augmented[col] += translation_vector[i % 3]
        for i, col in enumerate(z_columns):
            df_augmented[col] += translation_vector[i % 3]

    # Apply rotation around the Z-axis
    if rotation_angle != 0:
        angle_radians = np.radians(rotation_angle)
        cos_angle = np.cos(angle_radians)
        sin_angle = np.sin(angle_radians)

        for col in x_columns:
            y_col = col.replace('x', 'y')
            df_augmented[col], df_augmented[y_col] = (cos_angle * df_augmented[col] - sin_angle * df_augmented[y_col],
                                                      sin_angle * df_augmented[col] + cos_angle * df_augmented[y_col])
    
    # making the gesture index of the augment different to be added into the dataset back
    # will need to double up on the y train and test as well

    if "gesture_index" in df_augmented.columns:
        cur_time = time.time_ns()
        df_augmented["gesture_index"] += cur_time

    
    return df_augmented

def calculate_hand_motion_feature(df: pd.DataFrame, landmark_cols: list):
    df_copy = df.copy()
    print(df_copy.shape)
    s = time.process_time()
    df_elapsed = fe.calculate_elapsed_time(df_copy)    
    print(time.process_time()-s)

    s = time.process_time()
    df_temporal = fe.calculate_temporal_features(df_copy, landmark_cols)
    print(time.process_time()-s)
    # df_stats = fe.calculate_temporal_stats(df_copy, landmark_cols)
    # df_pairwise = fe.calculate_landmark_distances(df_copy, landmark_cols)
    # df_angle = fe.calculate_landmark_angles(df_copy, landmark_cols)
    # df_combined = pd.concat([df_copy, df_angle], axis=1)
    
    # Ensure there are no duplicate columns
    df_combined = df_copy.loc[:,~df_copy.columns.duplicated()]
    return df_combined


X_train_augmented = augment_model(X_train, noise_level=0.05, translation_vector=[0.6, -0.5, 0.05], rotation_angle=45)
X_test_augmented = augment_model(X_test, noise_level=0.05, translation_vector=[0.6, -0.5, 0.05], rotation_angle=45)

print(X_test.equals(X_test_augmented))


X_train_combined = pd.concat([X_train, X_train_augmented], axis=0, ignore_index=True)
X_test_combined = pd.concat([X_test, X_test_augmented], axis=0, ignore_index=True)


# print(X_train_combined.shape, X_train_augmented.shape, X_train.shape)
# print(X_test_combined.shape, X_test_augmented.shape, X_test.shape)
# print(y_train.shape, y_test.shape)

# print(X_train.dtypes)
# print(X_train_augmented.dtypes)
# print(X_train_combined.index.duplicated().sum())  # Should be 0
# print(X_train_combined.isnull().sum().sum())  # Should be 0
# print(np.isinf(X_train_combined).sum().sum())  # Should be 0

landmark_columns = [f"{col}" for col in dataframe.columns if col.startswith(("hx", "hy", "hz", "px", "py", "pz", "lx", "ly", "lz", "rx", "ry", "rz"))]
categorical_columns = ["gesture_index"]
numerical_columns = ["frame", "frame_rate", "frame_width", "frame_height"] + [f"{col}" for col in dataframe.columns if col.startswith("pose_visibility")]
derived_features =  ['elapsed_time'] + \
                    [f"{feat}_{col}" for feat in ["velocity", "acceleration", "jerk"] for col in landmark_columns if col.startswith(("lx", "ly", "lz", "rx", "ry", "rz"))]
time_series_columns = landmark_columns + derived_features     



res = [item for item in landmark_columns if item.startswith(("r", "l"))]
X_train_fe = calculate_hand_motion_feature(X_train_combined, res)
X_test_fe = calculate_hand_motion_feature(X_test_combined, res)


False
(16740, 1667) (8370, 1667) (8370, 1667)
(4140, 1667) (2070, 1667) (2070, 1667)
(8370, 3) (2070, 3)
hx_0             float32
hy_0             float32
hz_0             float32
hx_1             float32
hy_1             float32
                  ...   
frame_rate       float32
frame_width      float32
frame_height     float32
frame            float32
gesture_index      int64
Length: 1667, dtype: object
hx_0             float64
hy_0             float64
hz_0             float64
hx_1             float64
hy_1             float64
                  ...   
frame_rate       float32
frame_width      float32
frame_height     float32
frame            float32
gesture_index      int64
Length: 1667, dtype: object
0
0
0
(16740, 1667)
0.078125
109.125
(4140, 1667)
0.046875
29.015625


In [44]:
# code to reshape y, to have both the new augmented data and resgaoe shape the labesl
from sklearn.calibration import LabelEncoder
from keras._tf_keras.keras.utils import to_categorical


def reshape_y_labels(df: pd.DataFrame):
    
    df = pd.concat([df, df])
    print(df.shape)
    unique_sequences = []
    for _, group in df.groupby("gesture_index"):
        reset_points = group['frame'].diff().fillna(1) < 0
        if reset_points.any():
            unique_sequences.append(group[reset_points])
        else:
            # If no reset points, consider the whole group as unique
            unique_sequences.append(group.iloc[[0]])

    # Concatenate unique sequences
    df_unique = pd.concat(unique_sequences).reset_index(drop=True)
    return df_unique["gesture"]


y_train_reshaped = reshape_y_labels(y_train)
y_test_reshaped = reshape_y_labels(y_test)

print(y_train_reshaped.shape, y_test_reshaped.shape)

y_train_encoded, labels = pd.factorize(y_train_reshaped)
y_test_encoded, _ = pd.factorize(y_test_reshaped)
class_labels = labels

y_train_one_hot = to_categorical(y_train_encoded, num_classes=len(labels))
y_test_one_hot = to_categorical(y_test_encoded, num_classes=len(labels))



(16740, 3)
(4140, 3)
(279,) (69,)


In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PolynomialFeatures, PowerTransformer


def preprocess_pipeline(timeseries_columns, numerical_columns, categorical_columns):
    ts_numerical_transformer = Pipeline(steps=[
        ('imputer', KNNImputer(n_neighbors=5)), # might want to change this out back to the interpolatioon methods
        ('imputer2', SimpleImputer(strategy="mean")),
        ('scaler', MinMaxScaler())
        # ('smoother', FunctionTransformer(lambda x: x.rolling(window=3, min_periods=1).mean())),
        # ('differencing', FunctionTransformer(lambda x: x.diff().fillna(0))),
        # ('lag_features', FunctionTransformer(lambda x: pd.concat([x.shift(i) for i in range(1, 4)], axis=1).fillna(0))),
        # ('rolling_stats', FunctionTransformer(lambda x: pd.concat([x.rolling(window=3).mean(), x.rolling(window=3).std()], axis=1).fillna(0)))
    ])

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy="mean")),
        # ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Polynomial features
        # ('power', PowerTransformer(method='yeo-johnson')),   
        ("normalize", StandardScaler()),
        ('scaler', MinMaxScaler()),
        ('pca', PCA(n_components=10))
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy="most_frequent")), # technically this is wrong
        ("ohe", OneHotEncoder(sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('ts_num', ts_numerical_transformer, timeseries_columns),
            ('num', numerical_transformer, numerical_columns),
            ('cat', categorical_transformer, categorical_columns)
        ],
        remainder='passthrough',
        sparse_threshold=0,
        n_jobs=-1
    )
 
    preprocessor.set_output(transform="pandas")
    
    return preprocessor

preprocessor = preprocess_pipeline(time_series_columns, numerical_columns, categorical_columns)
X_train_transformed = preprocessor.fit_transform(X_train_fe)
X_test_transformed = preprocessor.transform(X_test_fe) # need to redo that gesture_index thing
print("done")

ValueError: Found unknown categories [1721617027697291200, 1721617035044262600, 1721617044341822500, 1721617053827646200, 1721617062115648000, 1721617070322155500, 1721617078635838400, 1721617087687420100, 1721617094888227100, 1721617102118686200, 1721617109386097100, 1721617116502778300, 1721617123704757000, 1721617130878078000, 1721617512655027300, 1721617518762291100, 1721617524726805800, 1721617530743470200, 1721617536743659800, 1721617542854370800, 1721617548850003800, 1721617555008531900, 1721617561073103200, 1721617567194924300, 1721617573423100100, 1721617579495190400, 1721617585481057400, 1721617591616257300, 1721617812796674500, 1721617819149660900, 1721617825538925900, 1721617831786486200, 1721617838053639900, 1721617844766273800, 1721617851086522600, 1721617857159002000, 1721617863553882100, 1721617869780379700, 1721617875838491100, 1721617882367760300, 1721617889361363800, 1721618180137850100, 1721618186665596700, 1721618192782339000, 1721618199059574000, 17216182059466257